# Exercises
Do your work for this exercise in a jupyter notebook named feature_engineering within the regression-exercises repo. Add, commit, and push your work.

1. Load the tips dataset.

    - Create a column named price_per_person. This should be the total bill divided by the party size.
    - Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
    - Use select k best to select the top 2 features for predicting tip amount. What are they?
    - Use recursive feature elimination to select the top 2 features for tip amount. What are they?
    - Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pydataset import data
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the tips dataset
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
# a. Create a column nacmes price_per_person. This should be the total bill divided by party size.
df['price_per_person'] = round((df.total_bill / df['size']), 2) 

In [4]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,6.15


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   size              244 non-null    int64  
 7   price_per_person  244 non-null    float64
dtypes: float64(3), int64(1), object(4)
memory usage: 17.2+ KB


In [6]:
df.day.value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [7]:
df['sex_encoded'] = df.sex.map({'Female':1, 'Male':0})
df['smoker_encoded'] = df.smoker.map({'Yes':1, 'No':0})
df['day_encoded'] = df.day.map({'Thur':4, 'Fri':5, 'Sat':6, 'Sun':7})
df['time_encoded'] = df.time.map({'Lunch':1, 'Dinner':0})
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,sex_encoded,smoker_encoded,day_encoded,time_encoded
1,16.99,1.01,Female,No,Sun,Dinner,2,8.49,1,0,7,0
2,10.34,1.66,Male,No,Sun,Dinner,3,3.45,0,0,7,0
3,21.01,3.50,Male,No,Sun,Dinner,3,7.00,0,0,7,0
4,23.68,3.31,Male,No,Sun,Dinner,2,11.84,0,0,7,0
5,24.59,3.61,Female,No,Sun,Dinner,4,6.15,1,0,7,0


In [8]:
df = df.drop(columns=['sex', 'smoker', 'day', 'time'])

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   size              244 non-null    int64  
 3   price_per_person  244 non-null    float64
 4   sex_encoded       244 non-null    int64  
 5   smoker_encoded    244 non-null    int64  
 6   day_encoded       244 non-null    int64  
 7   time_encoded      244 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 17.2 KB


In [16]:
train_validate, test = train_test_split(df, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)
train.shape, validate.shape, test.shape

((136, 8), (59, 8), (49, 8))

In [17]:
# split train into X, y 
X_train = train.drop(columns=['tip'])
y_train = train['tip']

X_validate = validate.drop(columns=['tip'])
y_validate = validate['tip']

X_test = test.drop(columns=['tip'])
y_test = test['tip']

X_train.head()

,total_bill,size,price_per_person,sex_encoded,smoker_encoded,day_encoded,time_encoded
19,16.97,3,5.66,1,0,7,0
173,7.25,2,3.62,0,1,7,0
119,12.43,2,6.22,1,0,4,1
29,21.70,2,10.85,0,0,6,0
238,32.83,2,16.42,0,1,6,0


b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
- I believe total bill will be the most important feature

In [18]:
# c. Use select k best to select the top 2 features for predicting tip amount. What are they?
kbest = SelectKBest(f_regression, k=2)
kbest.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7f89f9f898b0>)

In [19]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=X_train.columns)
kbest_results

,p,f
total_bill,7.186470e-20,115.984909
size,1.341642e-12,61.259089
price_per_person,1.306594e-03,10.783502
sex_encoded,2.844794e-01,1.154792
smoker_encoded,5.579978e-01,0.344909
day_encoded,1.045855e-01,2.670276
time_encoded,1.821449e-01,1.798647


In [20]:
X_train.columns[kbest.get_support()]

Index(['total_bill', 'size'], dtype='object')

In [21]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[kbest.get_support()]
)
X_train_transformed.head()

,total_bill,size
19,16.97,3.0
173,7.25,2.0
119,12.43,2.0
29,21.70,2.0
238,32.83,2.0


In [24]:
# d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [25]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train.columns)

,rfe_ranking
total_bill,1
size,3
price_per_person,2
sex_encoded,1
smoker_encoded,6
day_encoded,5
time_encoded,4


In [26]:
X_train.columns[rfe.get_support()]

Index(['total_bill', 'sex_encoded'], dtype='object')

In [27]:
X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[rfe.support_]
)
X_train_transformed.head()

,total_bill,sex_encoded
19,16.97,1.0
173,7.25,0.0
119,12.43,1.0
29,21.70,0.0
238,32.83,0.0


e. Why do you think select k best and recursive feature elimination might give different answers for the top features? 
- 
Does this change as you change the number of features your are selecting?
